# Sequence Analysis

In [1]:
import pandas as pd
from msresist.pre_processing import preprocessing
from msresist.sequence_analysis import EM_SeqClustering
import warnings
warnings.simplefilter("ignore")

In [2]:
pd.set_option('display.max_colwidth', 1000)

ABC = preprocessing(motifs=True, Vfilter=True, FCfilter=True, log2T=True)

In [3]:
ABC

,peptide-phosphosite,Master Protein Descriptions,PC9,Erl,R428,Erl/R428,Erl+HGF,Erl+FGF,Erl+IGF,KO Erl,KO R428,KO Erl/R428,position
1,---MRyVASYL,60S acidic ribosomal protein P2,-0.041348,0.330638,0.079619,-0.287413,-0.230024,-0.550611,-0.398655,-0.141238,-0.088036,1.327068,Y3-p
2,---MtyAYLFK,Ras-related protein Rab-2B,0.633928,0.259916,-0.055945,-0.434915,0.208498,0.418103,0.471005,-0.675985,-0.459938,-0.364667,Y3-p
3,-MAKAyDHLFK,Ras-related protein Rab-13,0.541208,0.090077,-0.229873,-1.142080,0.287358,-0.069477,1.225244,-0.160944,-0.093620,-0.447893,Y5-p
5,-MARDyDHLFK,Ras-related protein Rab-35,0.356559,-0.176429,0.267839,-0.796749,0.275687,-0.194960,0.501464,-0.143006,0.196517,-0.286921,Y5-p
8,AAAIAyGLDRT,Heat shock 70 kDa protein 1A,0.431886,0.894781,0.373969,-0.304444,-1.203818,-0.086804,-0.276191,-0.184870,-0.155607,0.511098,Y183-p
...,...,...,...,...,...,...,...,...,...,...,...,...,...
711,YNGDYyRQGRI,Tyrosine-protein kinase receptor UFO,2.000649,1.825741,1.407761,0.059403,1.791506,1.472838,1.270572,-3.221904,-3.181840,-3.424725,Y703-p
715,YRIDHyLGKEM,Glucose-6-phosphate 1-dehydrogenase,0.859847,0.526724,0.082206,-1.119890,0.295326,-0.272845,-0.389873,0.040738,0.442742,-0.464974,Y202-p
717,YSRAKsPQPPV,Heterogeneous nuclear ribonucleoprotein U,-1.506732,-0.552407,-0.764941,-0.304817,0.035451,-0.012596,-1.148686,0.686723,1.223962,2.344042,S271-p
718,YTGQPsPRGLH,Mitotic interactor and substrate of PLK1,0.560768,0.916321,0.153097,-0.712320,-0.328744,-0.341950,-0.130217,-0.060099,-0.526681,0.469825,S78-p


In [4]:
ncl, pYTS = 4, "Y"
clusters = EM_SeqClustering(ABC, ncl, pYTS, max_n_iter=30)

convergence has been reached at iteration 12


In [5]:
pd.set_option('display.max_rows', 100)
pd.DataFrame(clusters).T

,0,1,2,3
0,ACRAAYNLVRD,ASASEYAHRGL,GDYLSYREFHS,AKFINYVKNCF
1,AHDSLYRVHMP,GALADYEGKDV,QEEAEYVRALF,APHVHYARLKT
2,ARLGEYEDVSR,GCFDPYSDDPR,RVNEKYSTLPA,DARDLYDAGVK
3,ASQKDYSSGFG,GGSDNYGSLSR,SAQAQYDTPKA,DGENIYIRHSN
4,FHPEPYGLEDD,ISKQEYDESGP,YNGDYYRQGRI,HSDDLYAVPHR
5,FRDSTYDLPRS,NRGPAYGLSRE,AAAIAYGLDRT,EDLPLYQHQAT
6,KCVAPYPSLLS,PDRQAYEPPPP,AERDLYLENPE,EIKGFYDPPRR
7,KEVSTYIKKIG,SSESSYLSILP,ATKVVYSAPRS,FLRQLYEEEIR
8,KKPHRYRPGTV,AANPAYGRYNP,CKEAAYHPEVA,IKEKIYVLLRR
9,KKQKSYTLVVA,AGMTGYGMPRQ,DPSGTYHAWKA,MYQIKYEELQS
